In [1]:
import pandas as pd

In [2]:
"""
Loads in raw score file
Expecting the schema 
Name	Date	GF	GA	Team

Calcuates result, margies and goal difference
Does some prep work for Form and Win

Obviously names need to be consistant
"""

raw_data = pd.read_csv("../data/results_v2.csv")
raw_data["GD"] = raw_data["GF"] - raw_data["GA"]
raw_data.loc[(raw_data['GD'] == 0), 'GamePoints'] = 0.5
raw_data.loc[(raw_data['GD'] < 0), 'GamePoints'] = -1
raw_data.loc[(raw_data['GD'] > 0), 'GamePoints'] = 1
raw_data.loc[(raw_data['GD'] > 6), 'GamePoints'] = 1.5
raw_data.loc[(raw_data['GD'] < -6), 'GamePoints'] = -1.5
raw_data.loc[(raw_data['GamePoints'] >= 1), 'W'] = 1
raw_data.loc[(raw_data['GamePoints'] == 0.5), 'D'] = 1
raw_data.loc[(raw_data['GamePoints'] <= -1), 'L'] = 1
raw_data.loc[(raw_data['GamePoints'] == 1.5), 'MW'] = 1
raw_data.loc[(raw_data['GamePoints'] == -1.5), 'ML'] = 1

raw_data.loc[(raw_data['GamePoints'] >= 1), 'Result'] = '🔼'
raw_data.loc[(raw_data['GamePoints'] == 0.5), 'Result'] = '➖'
raw_data.loc[(raw_data['GamePoints'] <= -1), 'Result'] = '🔽'
raw_data.loc[(raw_data['GamePoints'] == 1.5), 'Result'] = '⏫'
raw_data.loc[(raw_data['GamePoints'] == -1.5), 'Result'] = '⏬'

raw_data = raw_data[raw_data["Name"].str.contains("\*")==False] # Removes Ringers

raw_data = raw_data.fillna(0)
pd.set_option('display.float_format', '{:.1f}'.format)
raw_data = raw_data.astype({'W':'int','D':'int','L':'int','MW':'int','ML':'int'})

raw_data['Date'] = pd.to_datetime(raw_data['Date'],dayfirst = True)
gameweek_int = raw_data.nunique().Date
gameweek = "Gameweek " + str(gameweek_int)

#display(raw_data)

In [3]:
"""
Creates the Form Column
"""
def form_column(raw_data):
    raw_data['game_rank'] = raw_data.sort_values("Date").groupby("Name").Date.rank(ascending=False)
    raw_data_form = raw_data.loc[raw_data['game_rank'] <= 5]
    form = raw_data_form.groupby(['Name'], as_index=False).agg({'Result': ' '.join})
    return form

form = form_column(raw_data)

#display(form)

In [4]:
"""
Creates Matches Played (MP), Partipation (Par%), Pts and Win% (W%)

It also aggreates all the raw data to create an up to date table
"""
def get_totals(raw_data):
    played = raw_data.groupby('Name').count()
    played = pd.DataFrame(played["Date"]).rename(columns={"Date":"MP"})
    data = raw_data.groupby('Name').sum(numeric_only=True).sort_values(by=['GamePoints'],ascending=False).join(played).rename(columns={"GamePoints":"Pts"})

    data["Par%"] = data["MP"].div(gameweek_int)*100
    data["W%"] = data["W"]/data["MP"]*100
    data = data.astype({'Par%':'int','W%':'int'})
    return data

totals = get_totals(raw_data)



#display(data)


In [5]:
"""
Join tables 
Drops unwanted columns 
Puts columns in the correct order
"""
table = totals.merge(form,on='Name').rename(columns={"Result":"Form"})
table.index = table.Name
table = table[['MP', 'W', 'D', 'L','MW', 'ML', 'GF', 'GA','GD','Par%','W%','Pts','Form']]
table.index.names = [gameweek]
table

,MP,W,D,L,MW,ML,GF,GA,GD,Par%,W%,Pts,Form
Gameweek 32,,,,,,,,,,,,,
Calz,18,11,1,6,5,1,278,227,51,56,61,7.5,⏫ ➖ 🔼 🔽 🔼
Reaper,26,16,0,10,3,5,366,365,1,81,61,5.0,🔼 🔽 🔼 ⏫ 🔼
Schurrle,20,12,0,8,4,2,305,275,30,62,60,5.0,🔼 🔼 🔽 ⏫ 🔽
Gazprom,21,12,0,9,2,3,294,294,0,65,57,2.5,🔼 🔼 🔽 🔽 🔼
Virlo,13,7,1,5,2,2,191,186,5,40,53,2.5,➖ 🔽 🔽 🔼 🔼
Zola,18,9,1,8,3,3,249,265,-16,56,50,1.5,🔽 🔽 🔼 🔼 ⏬
Keysey,13,6,1,6,2,1,188,173,15,40,46,1.0,🔽 🔽 🔽 ⏫ 🔼
Peggy,6,3,0,3,1,0,90,79,11,18,50,0.5,🔽 🔼 ⏫ 🔽 🔼
Gilet,16,7,0,9,3,3,226,239,-13,50,43,-2.0,🔽 🔼 🔽 🔼 ⏫


In [6]:
"""
TO DO

Add Graphics
Tracking player pairings
Tracking week by week table position

"""


'\nTO DO\n\nAdd Graphics\nTracking player pairings\nTracking week by week table position\n\n'

In [7]:
display(raw_data)

,Name,Date,GF,GA,Team,GD,GamePoints,W,D,L,MW,ML,Result,game_rank
0,Calz,2023-03-02,16,17,DARKS,-1,-1.0,0,0,1,0,0,🔽,18.0
1,Helmet,2023-03-02,16,17,DARKS,-1,-1.0,0,0,1,0,0,🔽,24.0
2,Kim,2023-03-02,16,17,DARKS,-1,-1.0,0,0,1,0,0,🔽,28.0
3,Mate,2023-03-02,16,17,DARKS,-1,-1.0,0,0,1,0,0,🔽,27.0
4,Peggy,2023-03-02,16,17,DARKS,-1,-1.0,0,0,1,0,0,🔽,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,Gazprom,2023-09-28,15,14,DARKS,1,1.0,1,0,0,0,0,🔼,1.0
315,Golfer,2023-09-28,14,15,BLANCOS,-1,-1.0,0,0,1,0,0,🔽,1.0
316,Snoop,2023-09-28,14,15,BLANCOS,-1,-1.0,0,0,1,0,0,🔽,1.0
317,Helmet,2023-09-28,14,15,BLANCOS,-1,-1.0,0,0,1,0,0,🔽,1.0


In [8]:
def count_same_team(df, name_col, date_col, team_col):
    count = 0
    for idx, row in df.iterrows():
        name = row[name_col]
        date = row[date_col]
        team = row[team_col]
        same_team_df = df[(df[team_col] == team) & (df[date_col] == date)]
        same_team_names = same_team_df[name_col].unique()
        if len(same_team_names) > 1 and name in same_team_names:
            print(same_team_names)
    return count

In [9]:
count_same_team(raw_data, 'Name', 'Date', 'Team')

['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Gazprom']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Gazprom']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Gazprom']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Gazprom']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Gazprom']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Calz' 'Kim' 'Virlo' 'Gazprom' 'Snoop']
['Calz' 'Kim' 'Virlo' 'Gazprom' 'Snoop']
['Calz' 'Kim' 'Virlo' 'Gazprom' 'Snoop']
['Calz' 'Kim' 'Virlo' 'Gazprom' 'Snoop']
['Calz' 'Kim' 'Virlo' 'Gazprom' 'Snoop']
['Gilet' 'Schurrle' 'Virlo' 'Gazprom']
['Gilet' 'Schurrle' 'Virlo' 'Gazprom']
['Gilet' 'Schurrle' 'Virlo' 'Gazprom']
['Gilet' 'Schurrle' 'Virlo' 

0